In [11]:
!pip install torch torchvision pillow pybboxes kaggle -q

In [12]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d andrewmvd/road-sign-detection
!mkdir "kaggle"
!mkdir "kaggle/input"
!unzip road-sign-detection.zip -d "kaggle/input/"

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [15]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [16]:
!mkdir "kaggle/working"
!cp -r yolov5 kaggle/working/

mkdir: cannot create directory ‘kaggle/working’: File exists


In [17]:
!pip install -r yolov5/requirements.txt -q

In [18]:
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
import pandas as pd

In [19]:
input_path = 'kaggle/input/'
output_path = 'kaggle/working/yolov5'

In [20]:
annotations_path = os.path.join(input_path, 'annotations')
annotations = os.listdir(annotations_path)

In [21]:
img_name_list = []
width_list = []
height_list = []
label_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []

for idx in tqdm(range(len(annotations))):

    tree = ET.parse(os.path.join(annotations_path, annotations[idx]))
    root = tree.getroot()

    img_name = root.find('filename').text

    size = root.find('size')
    width = size.find('width').text
    height = size.find('height').text

    for group in root.findall('object'):
        label = group.find('name').text
        bbox = group.find('bndbox')
        xmin = bbox.find('xmin').text
        ymin = bbox.find('ymin').text
        xmax = bbox.find('xmax').text
        ymax = bbox.find('ymax').text

        img_name_list.append(img_name)
        width_list.append(width)
        height_list.append(height)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        xmax_list.append(xmax)
        ymax_list.append(ymax)
        label_list.append(label)

100%|██████████| 877/877 [00:00<00:00, 6441.05it/s]


In [22]:
labels_df = pd.DataFrame({
                        'img_name': img_name_list,
                        'width': width_list,
                        'height': height_list,
                        'xmin': xmin_list,
                        'ymin': ymin_list,
                        'xmax': xmax_list,
                        'ymax': ymax_list,
                        'label': label_list})
labels_df.head()

,img_name,width,height,xmin,ymin,xmax,ymax,label
0,road747.png,300,400,235,99,299,171,speedlimit
1,road747.png,300,400,228,214,298,288,speedlimit
2,road529.png,300,400,170,151,249,249,speedlimit
3,road210.png,300,400,179,135,199,156,speedlimit
4,road498.png,300,400,86,130,118,171,speedlimit


In [23]:
import numpy as np
import pandas as pd
import os
import pybboxes as pbx
from collections import defaultdict
from tqdm import tqdm
import shutil

In [24]:
classes = labels_df['label'].unique().tolist()
classes

['speedlimit', 'trafficlight', 'crosswalk', 'stop']

In [25]:
## Add class number associated to classes
labels_df['class'] = labels_df['label'].apply(lambda x: classes.index(x))
labels_df.head()

,img_name,width,height,xmin,ymin,xmax,ymax,label,class
0,road747.png,300,400,235,99,299,171,speedlimit,0
1,road747.png,300,400,228,214,298,288,speedlimit,0
2,road529.png,300,400,170,151,249,249,speedlimit,0
3,road210.png,300,400,179,135,199,156,speedlimit,0
4,road498.png,300,400,86,130,118,171,speedlimit,0


In [26]:
## Generate dictionary where key is image_name and value is list of all bboxes inforamtion
img_dict = defaultdict(list)

for idx in tqdm(range(len(labels_df))):
    sample_label_list = []
    img_name = labels_df.loc[idx, 'img_name']
    xmin = labels_df.loc[idx, 'xmin']
    ymin = labels_df.loc[idx, 'ymin']
    xmax = labels_df.loc[idx, 'xmax']
    ymax = labels_df.loc[idx, 'ymax']
    class_num = labels_df.loc[idx, 'class']
    W, H = int(labels_df.loc[idx, 'width']), int(labels_df.loc[idx, 'height'])

    voc_bbox = (int(xmin), int(ymin) ,int(xmax), int(ymax))

    x_center, y_center, w, h = pbx.convert_bbox(voc_bbox, from_type="voc", to_type="yolo", image_size=(W,H))

    sample_label_list.append(str(class_num))
    sample_label_list.append(str(x_center))
    sample_label_list.append(str(y_center))
    sample_label_list.append(str(w))
    sample_label_list.append(str(h))
    line = ' '.join(sample_label_list)

    img_dict[img_name].append(line)

100%|██████████| 1244/1244 [00:00<00:00, 2578.39it/s]


In [27]:
# make labels dir in data folder of yolov5
labels_dir = f'{output_path}/data/labels'
if os.path.exists(labels_dir):
    shutil.rmtree(labels_dir)
os.mkdir(labels_dir)

In [28]:
## Generate .txt file for each image
for img_name, lines in img_dict.items():
    img_name = img_name.split('.')[0]
    with open(f'{labels_dir}/{img_name}.txt', 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

In [29]:
import os
import shutil
from random import shuffle

In [30]:
images_path = input_path + '/images'
labels_path = labels_dir   ## directory having labels in .txt format

In [31]:
# create train , val folders in data directory of yolov5
train_dir = output_path + '/data/train'
val_dir = output_path + '/data/val'

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

if os.path.exists(val_dir):
    shutil.rmtree(val_dir)

os.mkdir(train_dir)
os.mkdir(val_dir)

# train, val each containing images and labels folders
os.mkdir(train_dir + '/images')
os.mkdir(train_dir + '/labels')
os.mkdir(val_dir + '/images')
os.mkdir(val_dir + '/labels')

In [32]:
# Shuffle image file names before splitting into train and val data
files = os.listdir(images_path)
shuffle(files)

In [33]:
def split(files, ratio):
    elements = len(files)
    middle = int(elements * ratio)
    return [files[:middle], files[middle:]]

def copy_files(images_path, labels_path, destination_path, files):

    for file_name in files:
        file_name = file_name.split('.')[0]

        src = images_path + f'/{file_name}.png'
        dst = destination_path + '/images'
        shutil.copy(src, dst)

        src = labels_path + f'/{file_name}.txt'
        dst = destination_path + '/labels'
        shutil.copy(src, dst)

In [34]:
images_path, labels_path, train_dir

('kaggle/input//images',
 'kaggle/working/yolov5/data/labels',
 'kaggle/working/yolov5/data/train')

In [35]:
# Split and copy files in train and val folder
train_ratio = 0.75
train_files, val_files = split(files, train_ratio)

root = 'data/traffic_sign_data'

copy_files(images_path, labels_path, train_dir, train_files)
copy_files(images_path, labels_path, val_dir, val_files)

In [36]:
assert (len(os.listdir(train_dir + '/images')) + len(os.listdir(val_dir + '/images')) == len(os.listdir(images_path)))

In [37]:
with open(f'{output_path}/data/sign_data.yaml', 'w') as f:
    f.write('train: ../data/train/images\n')
    f.write('val: ../data/val/images\n')
    f.write('nc: 4\n')
    f.write(f"names: {classes}")

In [38]:
epochs = 20
!python /content/kaggle/working/yolov5/train.py --img 640 --batch 16 --epochs {epochs} --data sign_data.yaml --weights yolov5s.pt

2023-11-23 12:46:52.746728: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 12:46:52.746783: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 12:46:52.746827: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=sign_data.yaml, hyp=kaggle/working/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=F

In [39]:
# results stored in runs/train
# get the last stored result
exp = sorted(os.listdir(output_path + '/runs/train'))[-1]
exp_path = output_path + '/runs/train/' + exp

In [40]:
os.listdir(exp_path)

['hyp.yaml',
 'PR_curve.png',
 'val_batch2_labels.jpg',
 'P_curve.png',
 'F1_curve.png',
 'val_batch1_labels.jpg',
 'labels.jpg',
 'train_batch2.jpg',
 'train_batch1.jpg',
 'confusion_matrix.png',
 'results.csv',
 'labels_correlogram.jpg',
 'opt.yaml',
 'events.out.tfevents.1700743617.8caeadb9b73d.912.0',
 'R_curve.png',
 'val_batch0_labels.jpg',
 'val_batch2_pred.jpg',
 'weights',
 'results.png',
 'val_batch1_pred.jpg',
 'val_batch0_pred.jpg',
 'train_batch0.jpg']

In [44]:
!zip -r /content/trainexp.zip /content/kaggle/working/yolov5/runs/train

  adding: content/kaggle/working/yolov5/runs/train/ (stored 0%)
  adding: content/kaggle/working/yolov5/runs/train/exp/ (stored 0%)
  adding: content/kaggle/working/yolov5/runs/train/exp/hyp.yaml (deflated 45%)
  adding: content/kaggle/working/yolov5/runs/train/exp/PR_curve.png (deflated 18%)
  adding: content/kaggle/working/yolov5/runs/train/exp/val_batch2_labels.jpg (deflated 8%)
  adding: content/kaggle/working/yolov5/runs/train/exp/P_curve.png (deflated 10%)
  adding: content/kaggle/working/yolov5/runs/train/exp/F1_curve.png (deflated 8%)
  adding: content/kaggle/working/yolov5/runs/train/exp/val_batch1_labels.jpg (deflated 7%)
  adding: content/kaggle/working/yolov5/runs/train/exp/labels.jpg (deflated 31%)
  adding: content/kaggle/working/yolov5/runs/train/exp/train_batch2.jpg (deflated 4%)
  adding: content/kaggle/working/yolov5/runs/train/exp/train_batch1.jpg (deflated 4%)
  adding: content/kaggle/working/yolov5/runs/train/exp/confusion_matrix.png (deflated 27%)
  adding: conten